# 1. Linear Regression

In [1]:
import pandas as pd
import numpy as np

In [2]:
# https://taqm.epa.gov.tw/taqm/tw/HourlyData.aspx
raw_data = pd.read_csv('./data_Hsinchu.csv',
                 usecols=[0] + list(range(2, 27)),
                 na_values=['#', '*', 'x', 'NA', 'NR'],
                 encoding='big5').fillna(0)
data = raw_data.pivot(index='測項', columns='日期').T

$X=\begin{pmatrix} 1 & x_1 & \cdots & x_p \end{pmatrix}$

In [3]:
# append 'Intercept' column
Xpred = list(data.columns)
Xpred.remove('PM2.5')
Xcols = ['Intercept'] + Xpred

n = data.shape[0]
p = len(Xpred)
data = data.assign(Intercept=np.ones(n))

$y=X\hat\beta$

In [4]:
X = data[Xcols]
y = data[['PM2.5']]

### Coefficient
$\hat\beta=(X^tX)^{-1}X^ty$

In [5]:
invXtX = np.linalg.inv(np.dot(X.T, X))
beta_hat = np.dot(np.dot(invXtX, X.T), y)
beta_hat_v = beta_hat.reshape(18)

### Std. Error
$RSS=\displaystyle\sum_{i=1}^n(y_i-\hat y_i))^2$

$\hat\sigma^2=\frac{1}{n-p-1}RSS$

$\hat{SE}(\hat\beta)=\sqrt{\hat\sigma^2 diag((X^tX)^{-1})}$

In [6]:
rss = float(np.sum((y - np.dot(X, beta_hat)) ** 2))
sigma2_hat = rss  / (n - p - 1)
se_hat = np.sqrt(sigma2_hat * np.diag(invXtX))

### t-statistic
$t=\frac{\hat\beta}{\hat{SE}(\hat\beta)}$

In [7]:
tstat = beta_hat_v / se_hat

In [8]:
# result
pd.DataFrame({
    'Coefficient': beta_hat_v,
    'Std. Error': se_hat,
    't-statistic': tstat},
    index=Xcols)

,Coefficient,Std. Error,t-statistic
Intercept,-16.441063,2.788240,-5.896575
AMB_TEMP,-0.256375,0.029344,-8.736793
CH4,9.363683,3.081798,3.038383
CO,22.945558,1.225114,18.729323
NMHC,-7.449827,3.117487,-2.389690
NO,-0.103401,0.356399,-0.290126
NO2,0.234533,0.353019,0.664363
NOx,-0.276370,0.353746,-0.781268
O3,0.090552,0.009403,9.630123
PM10,0.240746,0.004270,56.378428


In [9]:
import itertools
p = 2
Xperm = list(itertools.permutations(Xpred, p))
coef_list = []
rss_list = []
invXtX_list = []
for Xp in Xperm:
    Xcols = ['Intercept'] + list(Xp)
    X = data[Xcols]
    invXtX = np.linalg.inv(np.dot(X.T, X))
    invXtX_list.append(invXtX)
    beta_tmp = np.dot(np.dot(invXtX, X.T), y)
    coef_list.append(beta_tmp)
    rss_list.append(float(np.sum((y - np.dot(X, beta_tmp)) ** 2)))
# argmin rss
idx = rss_list.index(min(rss_list))
features = list(Xperm[idx])
coef = coef_list[idx]
# std err
sigma2_hat = rss_list[idx]  / (n - p - 1)
se_hat = np.sqrt(sigma2_hat * np.diag(invXtX_list[idx]))
# t-statistic
tstat = coef.reshape(3) / se_hat
# result
pd.DataFrame({
    'Coefficient': coef.reshape(3),
    'Std. Error': se_hat,
    't-statistic': tstat},
    index=['Intercept'] + features)

,Coefficient,Std. Error,t-statistic
Intercept,-0.797494,0.301421,-2.645781
CO,19.731511,0.755315,26.123538
PM10,0.271850,0.004129,65.844846
